In [2]:
import numpy as np
import gmsh
from tqdm import tqdm


# High level meshing parameters
lc = 1000.0  # Mesh size
number_of_vertical_layers = 2 # for extrusion
stride_x=10 # Stride while reading surface/bed data
stride_y=10 # Stride while reading surface/bed data

def load_surface(filename, stride_x=10, stride_y=10):
    '''Load surface data from a file, subsampling it to reduce size.'''
    data = np.genfromtxt(filename, delimiter=",")
    data = data[data[:, 2] != -9999]

    # Find unique x and y values
    x_vals = np.unique(np.round(data[:, 0], 3))
    y_vals = np.unique(np.round(data[:, 1], 3))

    # Subsample using strides
    x_keep = set(x_vals[::stride_x])
    y_keep = set(y_vals[::stride_y])

    # Filter data
    data_strided = np.array([
        row for row in data
        if np.round(row[0], 3) in x_keep and np.round(row[1], 3) in y_keep
    ])

    return data_strided

top = load_surface("../Meshes/3d_meshing/AH_surf_xyz.dat",stride_x=stride_x, stride_y=stride_y)
bot = load_surface("../Meshes/3d_meshing/AH_bed_xyz.dat",stride_x=stride_x, stride_y=stride_y)

# Structured grid setup
x_vals = np.unique(np.round(top[:, 0], 3))
y_vals = np.unique(np.round(top[:, 1], 3))
nx, ny = len(x_vals), len(y_vals)
x_map = {x: i for i, x in enumerate(x_vals)}
y_map = {y: j for j, y in enumerate(y_vals)}

# Gridded z values
top_grid = np.full((ny, nx), np.nan)
bot_grid = np.full((ny, nx), np.nan)

for x, y, z in top:
    i, j = x_map[x], y_map[y]
    top_grid[j, i] = z

for x, y, z in bot:
    i, j = x_map[x], y_map[y]
    bot_grid[j, i] = z

# Start gmsh
gmsh.initialize()
gmsh.model.add("ExtrudedMesh")

point_tags = {}  # (i, j) → point tag

# Create bottom points
print("Creating bottom points...")
for j in tqdm(range(ny)):
    for i in range(nx):
        x = x_vals[i]
        y = y_vals[j]
        z = bot_grid[j, i]
        if not np.isnan(z):
            tag = gmsh.model.geo.addPoint(x, y, z, lc)
            point_tags[(i, j)] = tag

surface_count = 0
print("Creating extruded hex volumes...")
for j in tqdm(range(ny - 1)):
    for i in range(nx - 1):
        # Four bottom points for a quad
        corners = [(i, j), (i+1, j), (i+1, j+1), (i, j+1)]
        if not all(k in point_tags for k in corners):
            continue

        pts = [point_tags[k] for k in corners]
        lines = [
            gmsh.model.geo.addLine(pts[0], pts[1]),
            gmsh.model.geo.addLine(pts[1], pts[2]),
            gmsh.model.geo.addLine(pts[2], pts[3]),
            gmsh.model.geo.addLine(pts[3], pts[0]),
        ]
        cloop = gmsh.model.geo.addCurveLoop(lines)
        surf = gmsh.model.geo.addPlaneSurface([cloop])

        # Compute average extrusion height from top - bottom z
        top_z = [top_grid[k[1], k[0]] for k in corners]
        bot_z = [bot_grid[k[1], k[0]] for k in corners]
        if any(np.isnan(top_z)) or any(np.isnan(bot_z)):
            continue

        dz = np.mean([tz - bz for tz, bz in zip(top_z, bot_z)])

        # Extrude surface vertically by dz
        gmsh.model.geo.extrude([(2, surf)], 0, 0, dz, numElements=[number_of_vertical_layers], recombine=True)
        surface_count += 1

print(f"Extruded {surface_count} bottom quads into hex volumes.")

gmsh.model.geo.synchronize()
gmsh.model.mesh.generate(3)
gmsh.write("tests/AH_extruded_hexmesh.msh")
gmsh.finalize()

print("Mesh saved to tests/AH_extruded_hexmesh.msh")

Creating bottom points...


100%|██████████| 21/21 [00:00<00:00, 11444.96it/s]


Creating extruded hex volumes...


100%|██████████| 20/20 [00:07<00:00,  2.59it/s]


Extruded 430 bottom quads into hex volumes.
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 6 (Extruded)
Info    : [ 10%] Meshing curve 7 (Extruded)
Info    : [ 10%] Meshing curve 8 (Extruded)
Info    : [ 10%] Meshing curve 9 (Extruded)
Info    : [ 10%] Meshing curve 11 (Extruded)
Info    : [ 10%] Meshing curve 12 (Extruded)
Info    : [ 10%] Meshing curve 16 (Extruded)
Info    : [ 10%] Meshing curve 20 (Extruded)
Info    : [ 10%] Meshing curve 21 (Line)
Info    : [ 10%] Meshing curve 22 (Line)
Info    : [ 10%] Meshing curve 23 (Line)
Info    : [ 10%] Meshing curve 29 (Extruded)
Info    : [ 10%] Meshing curve 30 (Extruded)
Info    : [ 10%] Meshing curve 31 (Extruded)
Info    : [ 10%] Meshing curve 32 (Extruded)
Info    : [ 10%] Meshing curve 34 (Extruded)
Info    : [ 10%] Meshing curve 35 (Extruded)
Info    : [ 10%] Meshin